## Esercizio

Inizia da questo notebook.

Supponiamo che il prezzo della casa non dipenda solo dalle dimensioni, ma anche dal numero di finestre.

n_windows = np.random.randint(3,8,100)

house_prices = 100 + 0,2 * house_sizes - 0,00003 * house_sizes**2 + np.random.normal(0, 50, n_samples) + 15*n_windows

Prova:
1) modifica i dati iniziali, ma continua a usare solo house_sizes come regressore. Cosa è cambiato?
2) adatta il quaderno per usare sia house_sizes che n_windows come regressori. Cosa è cambiato?
3) trova un buon modo per visualizzare gli output della regressione lineare con 2 regressori



### **Esercizio 1: Cambiare i dati iniziali mantenendo solo house_sizes come regressore**  
Abbiamo questa nuova formula per generare i prezzi delle case:

\[
\text{house\_prices} = 100 + 0.2 \times \text{house\_sizes} - 0.00003 \times \text{house\_sizes}^2 + \text{rumore} + 15 \times \text{n\_windows}
\]

Ma nel primo esercizio **manteniamo solo house_sizes come regressore**, ignorando n_windows.  

👉 **Cosa pensi che succederà se nel modello non consideriamo il numero di finestre?**  
- Il modello sarà comunque in grado di stimare bene i prezzi delle case?  
- L'errore (MSE) aumenterà o diminuirà rispetto a prima?  
- Il modello sarà più soggetto a underfitting o overfitting?  




Abbiamo aggiunto **n_windows** (numero di finestre) come fattore che influenza il prezzo delle case, ma nel primo esercizio il nostro modello ignora questa variabile e usa solo **house_sizes** (dimensione della casa).  

### **Cosa potrebbe succedere?**  
1. **Il modello potrebbe essere meno preciso** 🤔  
   - Se il numero di finestre ha un impatto reale sul prezzo, ma il modello lo ignora, allora non riuscirà a spiegare bene tutti i dati.  
   - Ci aspettiamo che l'errore (MSE) **aumenti**, perché il modello non sta considerando un'informazione importante.  

2. **Maggiore varianza nei residui (differenze tra predetto e reale)** 📈  
   - Alcune case con la stessa dimensione potrebbero avere prezzi molto diversi a causa del numero di finestre.  
   - Se non consideriamo n_windows, il modello vedrà queste variazioni come "rumore casuale" invece che come una relazione spiegabile.  

3. **Potrebbe esserci underfitting** 📉  
   - Se il modello non riesce a spiegare bene i dati, potrebbe essere troppo semplice rispetto alla realtà.  

Ora facciamo un esperimento:  
🔹 **Alleniamo un modello solo con house_sizes** e vediamo i risultati!  


In [4]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
# Crea un set di dati campione (dimensioni della casa in piedi quadrati e prezzo in migliaia di dollari)
np.random.seed(42)  # For reproducible results
n_samples = 200

# Dimensioni delle case tra 1000 e 3000 piedi quadrati
house_sizes = np.random.randint(1000, 3000, n_samples)

# Crea il numero di finestre casualmente, come indicato nell'esercizio
n_windows = np.random.randint(3, 8, n_samples)

# Aggiunge il numero di finestre al calcolo del prezzo
house_prices = 100 + 0.2 * house_sizes - 0.00003 * house_sizes**2 + np.random.normal(0, 50, n_samples) + 15 * n_windows

# Crea il DataFrame
data = pd.DataFrame({
    'Size': house_sizes,
    'Price': house_prices
})

data.head()

,Size,Price
0,2126,500.535693
1,2459,478.990608
2,1860,368.117602
3,2294,529.312774
4,2130,417.321303


In [6]:

X = data[['Size']]  # Features (input)
y = data['Price']   # Target (output)

# Split  data training (80%) e testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea il modello di regressione lineare
model = LinearRegression()

# Alleniamo il modello
model.fit(X_train, y_train)

# Prediciamo i valori sui dati di test
y_pred = model.predict(X_test)

# Calcoliamo MSE, RMSE e R2 per valutare il modello
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Root Mean Squared Error
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared Score: {r2:.4f}")

Mean Squared Error: 2570.34
Root Mean Squared Error: 50.70
R-squared Score: 0.4739


modello precendete (lezione)= 
Mean Squared Error: 2670.55

Root Mean Squared Error: 51.68

R-squared Score: 0.4141

modello adesso = 
Mean Squared Error: 2570.34

Root Mean Squared Error: 50.70

R-squared Score: 0.4739

Ottimo lavoro! Vediamo insieme i risultati ottenuti e cosa ci dicono:

### **Modello precedente (senza finestre )**:
- **MSE = 2670.55**: L'errore quadratico medio è relativamente alto, indicando che il modello non è riuscito a catturare bene la variabilità dei dati.
- **RMSE = 51.68**: La radice dell'errore quadratico medio è anch'essa piuttosto alta, suggerendo che le previsioni non sono molto precise.
- **R² = 0.4141**: Il modello spiega solo il 41.41% della variabilità del prezzo delle case. Questo è un segno di **underfitting**, ovvero il modello è troppo semplice per rappresentare bene i dati.

### **Modello attuale (con finestre )**:
- **MSE = 2570.34**: L'errore è leggermente diminuito, ma non in modo drammatico. Questo suggerisce che, anche aggiungendo **n_windows**  nel calcolo del prezzo, il modello non sta riuscendo a catturare correttamente tutte le variabili.
- **RMSE = 50.70**: Anche questo valore è più basso rispetto al modello precedente, ma non di molto. Il modello è comunque impreciso.
- **R² = 0.4739**: L'R² è aumentato, indicando che il modello riesce a spiegare un po' più di variabilità nel prezzo delle case rispetto al modello precedente. Tuttavia, l'R² è ancora piuttosto basso, quindi il modello potrebbe essere ancora **sottodimensionato** (underfitting).

### **Cosa è cambiato?**
- **Miglioramento**: Sebbene l'aggiunta di finestre  non abbia portato a un miglioramento significativo, abbiamo visto un piccolo aumento nel punteggio **R²**, che suggerisce che il modello ora sta riuscendo a spiegare un po' più della variabilità del prezzo delle case.
- **Stato di underfitting**: Il modello è ancora molto semplice e non è riuscito a catturare completamente la complessità dei dati (relazione tra **house_sizes** e **n_windows**).

### **Conclusione dell'esercizio 1**:
Aggiungere il numero di finestre  ha migliorato un po' il modello, ma il risultato è ancora lontano dall'essere perfetto. Il modello **underfitta** ancora, perché usa solo la dimensione della casa e non considera altre variabili importanti come il numero di finestre. 

### **Prossimo passo**:
Nel prossimo esercizio, dovremo usare **sia house_sizes che n_windows** come regressori per vedere come cambia il modello. Questo ci permetterà di migliorare ulteriormente la previsione, riducendo l'errore e aumentando il punteggio **R²**.

Vuoi che passiamo direttamente all'esercizio 2, dove usiamo entrambe le variabili come regressori?

***Quando si  parlo di "rumore" ci si  riferisco alla parte aleatoria del calcolo del prezzo che abbiamo aggiunto nel nostro dataset per simulare una certa variabilità casuale. Nel nostro codice, quando abbiamo definito i prezzi delle case, abbiamo aggiunto un termine casuale che rappresenta un'incertezza o un disturbo casuale. Questo è il "rumore" a cui mi riferisco:***


house_prices = 100 + 0.2 * house_sizes - 0.00003 * house_sizes**2 + np.random.normal(0, 50, n_samples) + 15 * n_windows
La parte np.random.normal(0, 50, n_samples) genera un "rumore" che si distribuisce in modo casuale con una media di 0 e una deviazione standard di 50. Questo termine è introdotto per rendere il nostro modello più realistico, simulando il fatto che i prezzi delle case non dipendono solo dalla dimensione e dal numero di finestre, ma anche da altri fattori imprevedibili (come la posizione, lo stato della casa, e altri fattori economici).

***Cosa fa questo "rumore"?
Aggiunge variabilità casuale ai dati di prezzo.
Impedisce che il modello si adatti troppo perfettamente ai dati, rendendo l'analisi più simile a ciò che succede nel mondo reale (dove ci sono sempre fattori non misurabili che influenzano i risultati).
In sostanza, il "rumore" è solo un termine che simula una variabilità imprevedibile nei dati, e non è legato a nessuna delle variabili che stiamo cercando di modellare (come la dimensione della casa o il numero di finestre).***

# ESERCIZIO 2

Perfetto! Ora ci concentreremo sull'esercizio 2, dove dobbiamo adattare il notebook per utilizzare sia **house_sizes** che **n_windows** come regressori.

### **Cosa faremo?**
1. Utilizzeremo due variabili indipendenti (house_sizes e n_windows) per prevedere il prezzo della casa.
2. Confrontiamo il modello ottenuto con il modello precedente (che usava solo **house_sizes** come regressore).
3. Analizzeremo cosa cambia in termini di errori e accuratezza del modello.

### **Passo 1: Prepariamo i dati**

Abbiamo già creato il nostro dataset con le variabili **house_sizes** e **n_windows**. Ora, dobbiamo usarle entrambe come variabili indipendenti (regressori) per prevedere il **house_prices**.

### **Passo 2: Dividiamo i dati in set di addestramento e di test**
Anche in questo caso, dividiamo i dati in un set di addestramento (80%) e un set di test (20%) per allenare il modello e valutarne le performance su dati mai visti prima.

### **Passo 3: Crea e allena il modello**

Ora useremo una regressione lineare, ma con due variabili indipendenti. La funzione `LinearRegression()` può gestire più di una variabile indipendente (in questo caso, **house_sizes** e **n_windows**).

### **Passo 4: Valutiamo il modello**

Per vedere cosa è cambiato, calcoleremo le seguenti metriche:
- **MSE (Mean Squared Error)**: L'errore quadratico medio.
- **RMSE (Root Mean Squared Error)**: La radice dell'errore quadratico medio.
- **R² (R-squared)**: Il punteggio di determinazione che indica quanto bene il modello spiega la variabilità dei dati.

### **Passo 5: Confrontiamo con il modello precedente**

Con il modello che utilizza solo **house_sizes**, avevamo un certo valore di **MSE**, **RMSE**, e **R²**. Ora vediamo cosa cambia quando aggiungiamo **n_windows** come regressore.

### **Codice passo passo**

Ecco il codice che possiamo utilizzare per completare l'esercizio 2:

```python

data['n_windows'] = n_windows  # Aggiungiamo 'n_windows' al DataFrame

# Creiamo X (variabili indipendenti) utilizzando sia 'Size' che 'n_windows'
X = data[['Size', 'n_windows']]  # Ora entrambe le variabili sono nel DataFrame
y = data['Price']

# Suddividiamo i dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea il modello di regressione lineare
model = LinearRegression()

# Alleniamo il modello
model.fit(X_train, y_train)

# Prevediamo i valori
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calcoliamo gli errori
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Visualizziamo i risultati
print(f"Training MSE: {train_mse:.2f}")
print(f"Test MSE: {test_mse:.2f}")
print(f"Training RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")
print(f"Training R-squared: {train_r2:.4f}")
print(f"Test R-squared: {test_r2:.4f}")
```

### **Cosa cambia?**

1. **Aggiunta di un altro regressore**: Il modello ora tiene conto di più variabili indipendenti (sia **house_sizes** che **n_windows**). Questo lo rende più complesso, e ci aspettiamo che il modello migliori in termini di accuratezza, perché stiamo utilizzando più informazioni per fare le previsioni.
2. **Errori (MSE, RMSE)**: Ci aspettiamo che l'errore complessivo (MSE e RMSE) diminuisca rispetto al modello precedente (che usava solo **house_sizes**), poiché ora il modello ha più variabili su cui basare le previsioni.
3. **R²**: Il punteggio **R²** dovrebbe aumentare, il che indica che il modello spiega una percentuale maggiore della variabilità del prezzo delle case. Questo perché stiamo usando più variabili (entrambi i regressori) che sono più rilevanti.

### **Conclusioni**

Quando aggiungiamo **n_windows** come regressore, ci aspettiamo di vedere un miglioramento nelle metriche del modello (MSE, RMSE e R²), poiché stiamo usando una variabile aggiuntiva che influisce sul prezzo della casa. Quindi, il modello diventa più preciso nel fare le previsioni.

Prova a eseguire il codice e guarda cosa cambia nei risultati! Se vuoi, possiamo analizzare insieme i numeri che ottieni.

#
**Training MSE: 3041.11
Test MSE: 2456.97
Training RMSE: 55.15
Test RMSE: 49.57
Training R-squared: 0.4917
Test R-squared: 0.4971**

Ottimo! I risultati che ottieni indicano che l'aggiunta di **n_windows** come regressore ha migliorato le prestazioni del modello.

### **Analisi dei risultati**:
1. **MSE (Mean Squared Error)**:
   - **Training MSE**: 3041.11
   - **Test MSE**: 2456.97
   - Il MSE più basso nel test rispetto al training suggerisce che il modello sta generalizzando relativamente bene ai dati non visti (test set).

2. **RMSE (Root Mean Squared Error)**:
   - **Training RMSE**: 55.15
   - **Test RMSE**: 49.57
   - La differenza tra l'RMSE di training e test non è troppo elevata, il che è positivo. Il modello sembra essere abbastanza equilibrato.

3. **R-squared**:
   - **Training R-squared**: 0.4917
   - **Test R-squared**: 0.4971
   - Il valore di R-squared si è migliorato rispetto al modello precedente, indicando che ora il modello spiega meglio la variabilità dei prezzi delle case, sia sui dati di training che di test.

### **Cosa è cambiato rispetto al modello precedente?**
- **Prestazioni migliorate**: L'aggiunta di **n_windows** come variabile indipendente ha portato a un miglioramento delle metriche di performance (MSE, RMSE e R-squared).
- **Migliore capacità di previsione**: Il modello è ora in grado di utilizzare non solo la dimensione della casa, ma anche il numero di finestre per fare previsioni più accurate sui prezzi.

### **Prossimi passi**:
- Puoi provare a migliorare ulteriormente il modello, ad esempio testando altre variabili o algoritmi più complessi (ad esempio la regressione polinomiale).
- Se desideri visualizzare meglio i risultati, potresti anche creare un grafico di dispersione per esplorare come i due regressori influenzano i prezzi delle case.

Fammi sapere se vuoi continuare a esplorare o se hai altre domande!

## ESERCIZIO 3)
trovare un buon modo per visualizzare gli output della regressione lineare con 2 regressori

Per visualizzare i risultati di una regressione lineare con **due regressori** (in questo caso, **Size** e **n_windows**), possiamo adottare diverse tecniche di visualizzazione. Una delle opzioni più comuni è quella di utilizzare un grafico tridimensionale, dove possiamo rappresentare i due regressori sull'asse X e Y e la variabile dipendente (**Price**) sull'asse Z. Questo tipo di grafico ci permette di vedere come i due regressori influenzano il prezzo delle case e come il modello di regressione si adatta ai dati.

### **Passi per visualizzare i risultati**:

1. **Grafico 3D (scatter plot + superficie di regressione)**:
   - Useremo **matplotlib** per creare un grafico 3D.
   - Il grafico mostrerà i dati reali come punti (scatter plot) e la superficie del modello di regressione lineare.

2. **Creare una griglia per la previsione**:
   - Creiamo una griglia di valori per **Size** e **n_windows**.
   - Utilizzeremo il nostro modello di regressione per fare previsioni su questa griglia.
   
### Ecco il codice per visualizzare il modello di regressione lineare con due regressori:

```python
from mpl_toolkits.mplot3d import Axes3D

# Creiamo una griglia per i valori di 'Size' e 'n_windows'
size_range = np.linspace(data['Size'].min(), data['Size'].max(), 50)
nwindows_range = np.linspace(data['n_windows'].min(), data['n_windows'].max(), 50)

# Creiamo una meshgrid per i dati
Size_grid, NWindows_grid = np.meshgrid(size_range, nwindows_range)
X_grid = np.c_[Size_grid.ravel(), NWindows_grid.ravel()]  # Creiamo un array con le combinazioni di Size e n_windows

# Facciamo delle previsioni per la griglia
y_grid_pred = model.predict(X_grid)

# Rimodelliamo le previsioni per adattarle alla forma della griglia
y_grid_pred = y_grid_pred.reshape(Size_grid.shape)

# Creiamo un grafico 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Aggiungiamo il grafico scatter dei dati reali
ax.scatter(data['Size'], data['n_windows'], data['Price'], color='blue', alpha=0.5, label='Dati reali')

# Aggiungiamo la superficie di regressione
ax.plot_surface(Size_grid, NWindows_grid, y_grid_pred, color='red', alpha=0.5, label='Superficie di regressione')

# Etichette degli assi
ax.set_xlabel('Size (Square Feet)')
ax.set_ylabel('Number of Windows')
ax.set_zlabel('Price (Thousands of Dollars)')

# Titolo
ax.set_title('Regressione Lineare con 2 Regressori')

# Mostriamo il grafico
plt.legend()
plt.show()
```

### **Cosa fa il codice?**

1. **Crea una griglia di valori** per **Size** e **n_windows**. Utilizziamo `np.linspace` per creare una gamma di valori equidistanti per ciascun regressore.
2. **Crea una meshgrid** che combina tutte le combinazioni possibili di **Size** e **n_windows**.
3. **Previsioni**: Il modello viene utilizzato per prevedere il prezzo delle case per tutte le combinazioni nella griglia.
4. **Visualizzazione**:
   - I dati reali vengono rappresentati come punti blu (scatter plot).
   - La superficie del modello di regressione viene tracciata in rosso.

### **Interpretazione**:
- **Punti blu**: Rappresentano i dati reali delle case (dimensioni e numero di finestre).
- **Superficie rossa**: È la previsione fatta dal modello di regressione lineare, che mostra come il prezzo delle case varia in funzione della dimensione e del numero di finestre.

Questa visualizzazione ti consente di capire come il modello prevede il prezzo delle case in relazione a entrambi i regressori e come il modello si adatta ai dati.

Se vuoi esplorare ulteriori dettagli o modificare la visualizzazione, fammi sapere!

# 

ALTRE VISUALIZZAZIONI 



Certo! Per migliorare o modificare la visualizzazione, possiamo esplorare diverse opzioni. Ecco alcune idee su come possiamo adattare il grafico in modo da ottenere una visualizzazione più chiara e utile:

1. **Visualizzazione separata dei dati reali e della superficie di regressione**: Invece di combinare entrambe le visualizzazioni nello stesso grafico, possiamo visualizzare i dati reali (scatter plot) in un grafico e la superficie di regressione in un altro.

2. **Visualizzazione in 2D**: Se preferisci una visualizzazione più semplice, possiamo trarre un grafico 2D per ciascuna variabile indipendente separata, mostrando come la previsione cambia per un valore fisso dell'altra variabile.

3. **Modifica dei colori e degli stili**: Cambiare i colori, aggiungere trasparenze o utilizzare stili diversi per rendere la visualizzazione più chiara e facilmente leggibile.

### **1. Separare scatter plot e superficie di regressione**

In questa versione, mostrerò i dati reali come uno scatter plot e la superficie di regressione in un grafico separato.

```python
from mpl_toolkits.mplot3d import Axes3D

# Creiamo una griglia per i valori di 'Size' e 'n_windows'
size_range = np.linspace(data['Size'].min(), data['Size'].max(), 50)
nwindows_range = np.linspace(data['n_windows'].min(), data['n_windows'].max(), 50)

# Creiamo una meshgrid per i dati
Size_grid, NWindows_grid = np.meshgrid(size_range, nwindows_range)
X_grid = np.c_[Size_grid.ravel(), NWindows_grid.ravel()]  # Creiamo un array con le combinazioni di Size e n_windows

# Facciamo delle previsioni per la griglia
y_grid_pred = model.predict(X_grid)

# Rimodelliamo le previsioni per adattarle alla forma della griglia
y_grid_pred = y_grid_pred.reshape(Size_grid.shape)

# Creiamo due grafici 3D per separare i dati e la superficie di regressione
fig = plt.figure(figsize=(14, 7))

# Grafico 1: Dati reali
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(data['Size'], data['n_windows'], data['Price'], color='blue', alpha=0.5, label='Dati reali')
ax1.set_xlabel('Size (Square Feet)')
ax1.set_ylabel('Number of Windows')
ax1.set_zlabel('Price (Thousands of Dollars)')
ax1.set_title('Dati Reali')
ax1.legend()

# Grafico 2: Superficie di regressione
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(Size_grid, NWindows_grid, y_grid_pred, color='red', alpha=0.5, label='Superficie di regressione')
ax2.set_xlabel('Size (Square Feet)')
ax2.set_ylabel('Number of Windows')
ax2.set_zlabel('Price (Thousands of Dollars)')
ax2.set_title('Superficie di Regressione')
ax2.legend()

plt.tight_layout()
plt.show()
```

### **Cosa cambia?**
- **Grafico 1**: Mostra solo i dati reali delle case, cioè il scatter plot dei punti.
- **Grafico 2**: Mostra solo la superficie di regressione, che indica la previsione del modello.

### **2. Visualizzazione 2D per ciascun regressore (esempio per Size)**

Se preferisci una visualizzazione più semplice in 2D, possiamo esaminare come la previsione cambia per un valore fisso di **n_windows**. In questo esempio, vediamo come cambia il prezzo in funzione della **Size**, mantenendo fisso il numero di finestre.

```python
# Fissiamo il numero di finestre
fixed_nwindows = np.mean(data['n_windows'])

# Crea una gamma di valori per 'Size'
size_range = np.linspace(data['Size'].min(), data['Size'].max(), 100)

# Prevediamo i prezzi per ogni valore di 'Size' con n_windows fissato
X_fixed = np.c_[size_range, np.full_like(size_range, fixed_nwindows)]
y_pred = model.predict(X_fixed)

# Grafico 2D
plt.figure(figsize=(8, 6))
plt.plot(size_range, y_pred, color='red', label='Superficie di regressione (Size fissato)')
plt.scatter(data['Size'], data['Price'], color='blue', alpha=0.5, label='Dati reali')
plt.xlabel('Size (Square Feet)')
plt.ylabel('Price (Thousands of Dollars)')
plt.title('Relazione tra Size e Price (con n_windows fisso)')
plt.legend()
plt.show()
```

### **Cosa cambia?**
- In questo caso, stiamo visualizzando **solo** l'effetto di **Size** sul prezzo, mantenendo fisso il numero di finestre. La previsione viene calcolata per ogni valore di **Size**.

### **3. Cambiare il colore della superficie**

Possiamo anche cambiare il colore della superficie di regressione per renderla più visibile.

```python
# Creiamo una griglia per i valori di 'Size' e 'n_windows'
size_range = np.linspace(data['Size'].min(), data['Size'].max(), 50)
nwindows_range = np.linspace(data['n_windows'].min(), data['n_windows'].max(), 50)

# Creiamo una meshgrid per i dati
Size_grid, NWindows_grid = np.meshgrid(size_range, nwindows_range)
X_grid = np.c_[Size_grid.ravel(), NWindows_grid.ravel()]  # Creiamo un array con le combinazioni di Size e n_windows

# Facciamo delle previsioni per la griglia
y_grid_pred = model.predict(X_grid)

# Rimodelliamo le previsioni per adattarle alla forma della griglia
y_grid_pred = y_grid_pred.reshape(Size_grid.shape)

# Grafico 3D con colore personalizzato per la superficie
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Aggiungiamo la superficie di regressione con un colore personalizzato
ax.plot_surface(Size_grid, NWindows_grid, y_grid_pred, cmap='viridis', alpha=0.7)

# Aggiungiamo il grafico scatter dei dati reali
ax.scatter(data['Size'], data['n_windows'], data['Price'], color='blue', alpha=0.5, label='Dati reali')

# Etichette degli assi
ax.set_xlabel('Size (Square Feet)')
ax.set_ylabel('Number of Windows')
ax.set_zlabel('Price (Thousands of Dollars)')

# Titolo
ax.set_title('Regressione Lineare con 2 Regressori')

# Mostriamo il grafico
plt.legend()
plt.show()
```

### **Cosa cambia?**
- In questo caso, stiamo utilizzando la mappa di colori **'viridis'** per la superficie di regressione, che può migliorare la comprensione visiva delle variazioni nei dati.

Fammi sapere quale approccio ti sembra più utile o se hai altre idee su come personalizzare ulteriormente la visualizzazione!